In [5]:
import pandas as pd
import json
from pathlib import Path
# Model Algorithms
from sklearn.manifold import TSNE
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [6]:
ingredients = pickle.load(open("generated_data/cleaned_ingredients.pkl", 'rb'))
instructions = pickle.load(open("generated_data/instructions.pkl", 'rb'))


In [7]:
len(ingredients)

35829

In [8]:
len(instructions)

35829